In [1]:
from __future__ import print_function
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import time
import h5py
from pathlib import Path
import os

from art.attacks import CarliniL2Method, CarliniLInfMethod
from art.classifiers import PyTorchClassifier
from art.utils import load_cifar10
from art.attacks.evasion.projected_gradient_descent import ProjectedGradientDescent

%matplotlib inline
%config InlineBackend.figure_format='retina'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=128)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)
test_dataloader_single =  DataLoader(test_dataset, batch_size=1)

In [8]:
path = Path("C:/Users/Matach/OneDrive - Imperial College London/ICL/FYP/Experiments/saved_models/CIFAR10/Adversarial PGD")
file = "L2_1e-04.pth"
pretrained_model = os.path.join(path, file)
use_cuda = True

In [10]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
def train_classifier(cl, opt, x, y):
    x.to(device)
    y.to(device)
    opt.zero_grad()
    pred = cl(x)
    pred.to(device)
    err = F.nll_loss(F.log_softmax(pred, dim=0), y)
    err.backward()
    opt.step()
    return err, pred

def test_model(cl,test_loader): 
    correct = 0
    with torch.no_grad():
        cl.eval()
        for data, target in test_loader:
            output = cl(data.to(device))
            pred = output.data.max(1, keepdim=True)[1].to("cpu")
            target = np.argmax(target, axis=1)
            correct += pred.eq(target.data.view_as(pred)).sum()
        acc_test = float(correct.numpy() / len(test_loader.dataset))
        
    return acc_test

In [11]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
    
# Initialize the model.
model = Model().to(device)

# Load pre-trained model
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# Set the model in evaluation mode. In this case this is for the Dropout layers
print(model.eval())

# Load loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Make a classifier wrapper!
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

# Test model
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))

CUDA Available:  True
Model(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Dropout2d(p=0.05, inplace=False)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dr

In [12]:
# Create ART classifier 
def make_classifier(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    # Make a classifier wrapper!
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_, max_),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )
    return classifier


def adversarial_training_PGD(model, device, data, epsilon, alpha):
    dim = data.size()
    batch = torch.Tensor()
    fgsm = np.random.choice([0, 1], size=dim[0], p=[.5, .5])
    for image, is_fgsm in zip(data, fgsm):
        # VIP Reshape image for model compatibility
        image = image.view(1, dim[1], dim[2], dim[3])
        image, batch = image.to(device), batch.to(device)
        if is_fgsm:
            classifier = make_classifier(model)
            adv_crafter = ProjectedGradientDescent(classifier, norm=np.inf, eps=epsilon, eps_step=alpha, max_iter=7)
            perturbed_image = adv_crafter.generate(x=image.cpu().detach().numpy())
            batch = torch.cat((batch, torch.Tensor(perturbed_image).to(device)), dim=0)
        else:
            batch = torch.cat((batch, image), dim=0)
    return batch

In [6]:
path = Path("C:/Users/Matach/OneDrive - Imperial College London/ICL/FYP/Experiments/saved_models/CIFAR10/Adversarial PGD")
file = "L2_1e-04.pth"

data = np.vstack((x_train, x_test))
cifar10_std = data.std()
eps = (4 / 255.) / cifar10_std
alpha = (2 / 255.) / cifar10_std
print(eps, alpha)

0.06236745581885411 0.031183727909427056


In [7]:
num_epochs = 20
e_losses = [] 

# Load pre-trained model
cl = Model().to(device)
# cl = model

# optimizer
cl_opt = optim.Adam(cl.parameters(), lr=0.001, weight_decay=1e-04)
criterion = nn.CrossEntropyLoss()

for e in range(num_epochs):
    cl.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        target = np.argmax(target, axis=1)  # transform from one-hot to int
        new_batch = adversarial_training_PGD(cl, device, data, epsilon=eps, alpha=alpha)
        c_error,c_pred = train_classifier(cl,cl_opt, new_batch.to(device), target.to(device))
        e_losses.append(c_error.cpu().data.numpy())
    acc = test_model(cl,test_dataloader)
    torch.save(cl.state_dict(), '{name}_{epoch}'.format(name=os.path.join(path, file), epoch=e)) # save file
    print("Epoch ", e, ": Test accuracy: ", 100*acc, "%")

print("Performance in the trained model: ")
acc_test = test_model(cl,test_dataloader)
print("Test accuracy: ", 100*acc_test, "%")
print("FINISHED!!!")

Epoch  0 : Test accuracy:  44.35 %
Epoch  1 : Test accuracy:  48.089999999999996 %
Epoch  2 : Test accuracy:  52.78 %
Epoch  3 : Test accuracy:  56.67 %
Epoch  4 : Test accuracy:  57.91 %
Epoch  5 : Test accuracy:  62.71 %
Epoch  6 : Test accuracy:  61.9 %
Epoch  7 : Test accuracy:  63.38 %
Epoch  8 : Test accuracy:  63.18 %
Epoch  9 : Test accuracy:  64.28 %
Epoch  10 : Test accuracy:  65.88000000000001 %
Epoch  11 : Test accuracy:  65.23 %
Epoch  12 : Test accuracy:  65.81 %
Epoch  13 : Test accuracy:  65.25999999999999 %
Epoch  14 : Test accuracy:  65.86999999999999 %
Epoch  15 : Test accuracy:  65.58 %
Epoch  16 : Test accuracy:  66.34 %
Epoch  17 : Test accuracy:  65.68 %
Epoch  18 : Test accuracy:  66.22 %
Epoch  19 : Test accuracy:  65.45 %
Performance in the trained model: 
Test accuracy:  65.45 %
FINISHED!!!
